In [1092]:
using LinearAlgebra
using RowEchelon
using Plots

In [1093]:
m = 5
S = rand(m,m)
R = rand(m,m)

5×5 Matrix{Float64}:
 0.722153    0.47156   0.150988  0.9966    0.721241
 0.00197479  0.803543  0.543133  0.919352  0.544967
 0.361652    0.344518  0.848086  0.475725  0.494888
 0.232953    0.605542  0.45116   0.201272  0.782097
 0.270555    0.352335  0.593914  0.509349  0.734402

In [1094]:
E_val, E_vec = eigen(S,R)

GeneralizedEigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
5-element Vector{ComplexF64}:
 -0.5357209717350414 + 0.0im
 0.13403300297035337 - 0.5208139903457869im
  0.1340330029703534 + 0.5208139903457869im
  0.8431769087820661 + 0.0im
   2.727783719883253 + 0.0im
vectors:
5×5 Matrix{ComplexF64}:
  0.699608+0.0im   0.396713+0.603287im   …   0.665678+0.0im       -1.0+0.0im
 -0.151899+0.0im  -0.579676+0.229838im       0.923188+0.0im  -0.880637+0.0im
   0.71378+0.0im   0.272807-0.370051im            1.0+0.0im  -0.242496+0.0im
      -1.0+0.0im   0.535635-0.0431843im     -0.212674+0.0im   0.561856+0.0im
  0.137766+0.0im  -0.765146+0.0971636im      0.460272+0.0im     0.9015+0.0im

In [1095]:
E_val, E_vec = eigen(inv(R)*S)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
5-element Vector{ComplexF64}:
 -0.5357209717350399 + 0.0im
 0.13403300297035364 - 0.5208139903457869im
 0.13403300297035364 + 0.5208139903457869im
  0.8431769087820659 + 0.0im
  2.7277837198832513 + 0.0im
vectors:
5×5 Matrix{ComplexF64}:
  -0.489705+0.0im  -0.224253-0.457932im  …   0.416661+0.0im  -0.580972+0.0im
   0.106325+0.0im   0.426546-0.109446im      0.577841+0.0im  -0.511626+0.0im
  -0.499625+0.0im  -0.224039+0.234974im       0.62592+0.0im  -0.140884+0.0im
   0.699971+0.0im  -0.379087-0.017398im     -0.133117+0.0im   0.326423+0.0im
 -0.0964319+0.0im   0.544676-0.0im           0.288093+0.0im   0.523746+0.0im

^ Eigenvalues are identical, but vectors are scaled slightly differently. I *think* this is related to inaccuracy in the computation of inv(). Trying a version with integer values below.

In [1096]:
m = 2
S = rand(m,m)*10
R = rand(m,m)*10

2×2 Matrix{Float64}:
 2.93511   0.363455
 0.780364  6.13672

To get well-behaved reconstruction:

S = [7.0 6.0; 6.0 5.0]

R = [9.0 1.0; 5.0 4.0]

In [1097]:
E_val, E_vec = eigen(S,R)

GeneralizedEigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 -1.6164235956548032
  2.6892906119134308
vectors:
2×2 Matrix{Float64}:
 -1.0       1.0
  0.647291  0.692798

In [1098]:
E_val_i, E_vec_i = eigen(inv(R)*S)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
2-element Vector{Float64}:
 -1.6164235956548034
  2.6892906119134303
vectors:
2×2 Matrix{Float64}:
 -0.839481  -0.822004
  0.543388  -0.569482

Interesting. However might be fine because relative contribution remains the same. Let's check below by trying to reconstruct the matrices via eigenlayers.

In [1099]:
norm = sqrt(E_vec[:,1]'E_vec[:,1])

1.1912116154609873

In [1100]:
norm_i = sqrt(E_vec_i[:,1]'E_vec_i[:,1])

1.0

The eigenvectors from eigen(S,R) are not normalized. Normalizing them below:

In [1101]:
for ii in 1:m
    norm = sqrt(E_vec[:,ii]'E_vec[:,ii])
    E_vec[:,ii] ./= norm 
end

In [1102]:
norm = sqrt(E_vec[:,1]'E_vec[:,1])

1.0

In [1103]:
rebuild = zeros(m,m)

for ii in 1:m
    rebuild += E_vec[:,ii] .* E_val[ii] * E_vec[:,ii]'
end

In [1104]:
rebuild_i = zeros(m,m)

for ii in 1:m
    rebuild_i += E_vec_i[:,ii] .* E_val_i[ii] * E_vec_i[:,ii]'
end

In [1105]:
real(rebuild)

2×2 Matrix{Float64}:
 0.677986  1.99626
 1.99626   0.394881

In [1106]:
real(rebuild_i)

2×2 Matrix{Float64}:
 0.677986  1.99626
 1.99626   0.394881

In [1107]:
inv(R)*S

2×2 Matrix{Float64}:
 0.463326  3.21301
 1.44085   0.609541

In [1108]:
inv(R)*S ./ rebuild

2×2 Matrix{Float64}:
 0.683386  1.60952
 0.721774  1.54361

Hooray!

The discrepancy was simply due to the fact that generalized eigendecomposition in Julia doesn't return unit length eigenvectors. <- This is wrong I think>